# Python常见高级应用

这部分主要参考[python3-cookbook](https://python3-cookbook.readthedocs.io/zh_CN/latest/index.html)，主要记录一些在实际编写模型算法及开发应用程序的过程中值得一用的高级技巧。

结合basic-python的结构，针对每一部分都有高级的用法，主要分为以下部分：

- 基础语法进阶：比如结合基本数据结构的基本算法，文本及字符串处理，迭代器、生成器等；
- 函数与类的高级运用：比如元编程；
- python实际应用：从项目构建的模块、包等概念到常见应用，如脚本系统管理、web编程、并发管理、测试调试等。

## 基础语法进阶

### 字符串与文本

#### 字符串中插入变量

想创建一个内嵌变量的字符串，变量被它的值所表示的字符串替换掉。这在包括print结果，构建带参数的url等很多场景下都会用到。

Python并没有对在字符串中简单替换变量值提供直接的支持。 但是通过使用字符串的 format() 方法来解决这个问题。比如：

In [1]:
s = '{name} has {n} messages.'
s.format(name='Guido', n=37)

'Guido has 37 messages.'

或者，如果要被替换的变量能在变量域中找到， 那么可以结合使用format_map() 和 vars() 。就像下面这样：

In [2]:
name = 'Guido'
n = 37
s.format_map(vars())

'Guido has 37 messages.'

vars() 还有一个有意思的特性就是它也适用于对象实例。

In [5]:
class Info:
     def __init__(self, name, n):
            self.name = name
            self.n = n
            
a = Info('Guido',37)
s_out=s.format_map(vars(a))
print(s_out)

Guido has 37 messages.


format 和 format_map() 的一个缺陷就是它们并不能很好的处理变量缺失的情况，一种避免这种错误的方法是另外定义一个含有 __missing__() 方法的字典对象，就像下面这样：

In [7]:
class safesub(dict):
# """防止key找不到"""
    def __missing__(self, key):
        return '{' + key + '}'
    
del n # Make sure n is undefined
s.format_map(safesub(vars()))

'Guido has {n} messages.'

多年以来由于Python缺乏对变量替换的内置支持而导致了各种不同的解决方案。比如常见的%做占位符。不过format() 和 format_map() 相比较上面这些方案而已更加先进，因此应该被优先选择。 使用 format() 方法还有一个好处就是你可以获得对字符串格式化的所有支持(对齐，填充，数字格式化等待)， 而这些特性是使用像模板字符串之类的方案不可能获得的。

In [8]:
s = '{name} has {n} messages，{name}.'
s.format(name='Guido', n=37)

'Guido has 37 messages，Guido.'



## 函数与类高阶

## python实战

### 脚本编程与系统管理

#### 命令行解析器

Python 命令行与参数解析方法有很多工具，这里参考python[官方文档](https://docs.python.org/zh-cn/3/library/argparse.html)
和[blog](https://zhuanlan.zhihu.com/p/34395749)学习使用python 自带的argparse ，来说明python 如何进行命令行解析。

通俗来说，命令行与参数解析就是当你输入cmd 打开dos 交互界面时候，启动程序要进行的参数给定。比如在dos 界面输入：

```code
python openPythonFile.py "a" -b "number"
```

其中，"a" -b等就是命令行与参数解析要做的事情。

就是设计程序在运行时必须给定某些额外参数才能运行，也就是如果设置了命令行参数解析，那么各种编译器按F5 是无法直接运行程序的。

用途就是不能随便就能运行脚本，可以达到一定程度上的安全功能。

总体上分为三大步：

- 创建解析
- 添加参数
- 解析参数

使用 argparse 的第一步是创建一个 ArgumentParser 对象。

ArgumentParser 对象包含将命令行解析成 Python 数据类型所需的全部信息。

In [ ]:
import argparse
parser = argparse.ArgumentParser(description='Process some integers.')

第二步是添加参数，给一个 ArgumentParser 添加程序参数信息是通过调用 add_argument() 方法完成的。

通常，这些调用指定 ArgumentParser 如何获取命令行字符串并将其转换为对象。这些信息在 parse_args() 调用时被存储和使用。

add_argument方法定义单个的命令行参数应当如何解析。每个形参都在下面有它自己更多的描述：

- name or flags - 一个命名或者一个选项字符串的列表，例如 foo 或 -f, --foo。
- action - 当参数在命令行中出现时使用的动作基本类型。
- nargs - 命令行参数应当消耗的数目。
- const - 被一些 action 和 nargs 选择所需求的常数。
- default - 当参数未在命令行中出现时使用的值。
- type - 命令行参数应当被转换成的类型。
- choices - 可用的参数的容器。
- required - 此命令行选项是否可省略 （仅选项可用）。
- help - 一个此选项作用的简单描述。
- metavar - 在使用方法消息中使用的参数值示例。
- dest - 被添加到 parse_args() 所返回对象上的属性名。大多数ArgumentParser actions增加一些值作为the object的一个属性被parse_args()返回 

In [ ]:
parser.add_argument('integers', metavar='N', type=int,
                    nargs='+', help='an integer for the accumulator')
parser.add_argument('--sum', dest='accumulate', action='store_const',
                    const=sum, default=max,
                    help='sum the integers (default: find the max)')

最后ArgumentParser 通过 parse_args() 方法解析参数。它将检查命令行，把每个参数转换为适当的类型然后调用相应的操作。

在大多数情况下，这意味着一个简单的 Namespace 对象将从命令行参数中解析出的属性构建。

在脚本中，通常 parse_args() 会被不带参数调用，而 ArgumentParser 将自动从 sys.argv 中确定命令行参数。

In [ ]:
args=parser.parse_args(['--sum', '7', '-1', '42'])
print(args.accumulate(args.integers))

### 测试、调试与异常

In [ ]:
import inc_dec    # The code to test


def test_increment():
    assert inc_dec.increment(3) == 4


def test_decrement():
    assert inc_dec.decrement(3) == 2